## 缺口填充

模型缺口填充的任务是找出必须将哪些反应添加到模型中以使其可行。已经报道了几种这样的算法，例如[Kumar等人，2009](http://dx.doi.org/10.1371/journal.pcbi.1000308)和[Reed等人，2006](http://www.pnas.org/content/103/46/17480.short)。cobry有一个缺口填充实现，与Reed等人的实现非常相似，我们使用混合整数线性程序来计算需要为用户定义的反应集合添加的最小反应数量，即通用模型。简单地说，就是我们尝试的问题

Minimize: $$\sum_i c_i * z_i$$

subject to

$$Sv = 0$$
$$v^\star \geq t$$
$$l_i\leq v_i \leq u_i$$
$$v_i = 0 \textrm{ if } z_i = 0$$

其中*l*， *u*分别是*i*反应的下界和上界，*z*是一个指标变量，如果不使用该反应则为零，否则为1，*c*是与使用*i*反应相关的用户定义成本，$v^\star$是物镜的通量，*t*是该物镜的下界。为了证明这一点，让我们以一个模型为例，并从中移除一些基本的反应。

In [2]:
from cobra.io import load_model
from cobra.flux_analysis import gapfill
model = load_model("iYS1720")

在这个模型中，d -果糖-6-磷酸是必需的代谢物。我们将移除所有使用它的反应，并把它们放到一个单独的模型中。

In [3]:
import cobra
universal = cobra.Model("universal_reactions")
for i in [i.id for i in model.metabolites.f6p_c.reactions]:
    reaction = model.reactions.get_by_id(i)
    universal.add_reactions([reaction.copy()])
    model.remove_reactions([reaction])

现在，因为这些空白，所以模型不会发展。

In [4]:
model.optimize().objective_value

0.0

我们将使用模型的原始目标，生长，来找出哪些被移除的反应是模型再次可行所必需的。这与[Kumar et al. 2009](http://dx.doi.org/10.1371/journal.pcbi.1000308)的“不增长但增长(NGG)”预测非常相似。

In [5]:
solution = gapfill(model, universal, demand_reactions=False)
for reaction in solution[0]:
    print(reaction.id)

GF6PTA
TALA


通过多次迭代，我们可以得到多个可能的反应集。

In [6]:
result = gapfill(model, universal, demand_reactions=False, iterations=4)
for i, entries in enumerate(result):
    print("---- Run %d ----" % (i + 1))
    for e in entries:
        print(e.id)

---- Run 1 ----
GF6PTA
TALA
---- Run 2 ----
GF6PTA
TALA
---- Run 3 ----
GF6PTA
PGI
TKT2
---- Run 4 ----
GF6PTA
TALA


我们也可以不使用最初的目标，指定一种我们希望模型能够产生的代谢物。

In [7]:
with model:
    model.objective = model.add_boundary(model.metabolites.f6p_c, type='demand')
    solution = gapfill(model, universal)
    for reaction in solution[0]:
        print(reaction.id)

PGI


最后，请注意，使用混合整数线性规划在计算上相当昂贵，对于较大的模型，您可能需要考虑替代的[间隙填充方法](http://opencobra.github.io/cobrapy/tags/gapfilling/)和[重建方法](http://opencobra.github.io/cobrapy/tags/reconstruction/)。